# Exploring the Model Control Plane Backend: A Practical Guide
Welcome to our tour into the Model Control Plane, where we'll focus on two independent pipelines within ZenML. Each pipeline works on its own, creating specific artifacts. But what's fascinating is that these seemingly separate pipelines are intricately connected, all with the goal of delivering precise predictions.

Before the Model Control Plane, connecting these pipelines and consolidating everything was a challenge. Imagine extracting a trained model artifact from the training pipeline and smoothly integrating it into the predictions pipeline. Previously, this involved complex ID references, leading to constant config updates, or blindly relying on the latest training run. But what if that run didn't meet the necessary performance standards? Using a subpar model for predictions was out of the question, especially for vital applications!

Enter the Model Control Plane. This feature empowers you to effortlessly group pipelines, artifacts, and crucial business data into a unified entity: a `Model`. A Model captures lineage information and more. Within a Model, different `Model Versions` can be staged. For example, you can rely on your predictions at a specific stage, like `Production``, and decide whether the Model Version should be promoted based on your business rules during training. Plus, accessing data from other Models and their Versions is just as simple, enhancing the system's adaptability.

## Overview of the process

![Pipelines Overview](_assets/train_prediction_example.png)

Each time the `train_and_promote` pipeline runs, it creates a new iris_classifier. However, it only promotes the created model to `Production` if a certain accuracy threshold is met. The `do_predictions` pipeline simply picks up the latest Promoted model and runs batch inference on it. That way these two pipelines can independently be run, but can rely on each others output.

## Installation and Initialization


In [1]:
from zenml.environment import Environment

if Environment.in_google_colab():
    # Install Cloudflare Tunnel binary
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudf

### Warning: This will remove all your local ZenML artifacts. If you don't want this then just remove the zenml clean command

In [6]:
!pip install "zenml[server,dev]>=0.45.5"
!zenml clean -y  # remove this if you dont want to lose local data
!zenml integration install sklearn -y

import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True)


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
The local ZenML dashboard is not running.
Deleted local ZenML config from 
/home/htahir1/workspace/zenml_io/zenml_playground/zenml-plugins/model_control_pl
ane/.zen.
Deleting 'local_stores' directory from global config.
Deleted global ZenML config from /home/htahir1/.config/zenml.
Creating default workspace 'default' ...
The current repo active workspace is no longer available. Resetting the active workspace to 'default'.
The current repo active stack is no longer available. Resetting the active stack to default.
Traceback (most recent call last):
  File "/home/htahir1/.virtualenvs/demo/lib/python3.8/site-packages/zenml/zen_stores/base_zen_store.py", line 418, in validate_active_config
    active_stack = self.get_stack(stack_id=active_stack_id)
  File "/home/htahir1/.virtualenvs/demo/lib/python3.8/site-packages/zenml/zen_stores/sql_zen_store.py", line 1100, in get_stack
    

{'status': 'ok', 'restart': True}

In [14]:
!rm -rf .zen
!zenml init

⠋ Initializing ZenML repository at 
/home/htahir1/workspace/zenml_io/zenml_playground/zenml-plugins/model_control_pl
ane.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
ZenML repository initialized at 
/home/htahir1/workspace/zenml_io/zenml_playground/zenml-plugins/model_control_pl
ane.
⠋ Initializing ZenML repository at 
/home/htahir1/workspace/zenml_io/zenml_playground/zenml-plugins/model_control_pl
ane.
⠋ Initializing ZenML repository at 
/home/htahir1/workspace/zenml_io/zenml_playground/zenml-plugins/model_control_pl
ane.

The local active stack was initialized to 'default'. This local configuration 
will only take effect when you're running ZenML from the initialized repository 
root, or from a subdirectory. For more information on repositories and 
configurations, please visit 
https://docs.zenml.io/user-guide/starter-guide/understand-stacks.


In [19]:
!zenml model list

No models found.


In [20]:
from zenml import __version__

__version__

'0.45.5'

## Training Pipeline


In [18]:
from zenml import pipeline
from zenml.model import ModelConfig

from steps.train.load import load_data
from steps.train.promote import promote_model
from steps.train.train import train_and_evaluate


@pipeline(
    enable_cache=False,
    model_config=ModelConfig(
        name="iris_classifier",
        license="Apache",
        description="Show case Model Control Plane.",
        create_new_model_version=True,
        delete_new_version_on_failure=True,
    ),
)
def train_and_promote_model():
    train_data, test_data = load_data()
    _, score = train_and_evaluate(train_data=train_data, test_data=test_data)
    promote_model(score=score)

train_and_promote_model()

Creation of new model version was requested, but no version name was explicitly provided. Setting version to running.
Initiating a new run for the pipeline: train_and_promote_model.


AttributeError: 'NoneType' object has no attribute 'to_model'